Подключение библиотек

In [ ]:
import pandas as pd
import cx_Oracle
from sqlalchemy import create_engine 

Настройка подключения к базе данных

In [ ]:
db_config = {
    'user': '321', # имя пользователя
    'pwd': '123456', # пароль
    'host': 'DWH',
    'port': 1521, # порт подключения
    'db': 'DWH' # название базы данных
}  

connection_string = 'oracle://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

Подключение к базе

In [ ]:
engine = create_engine(connection_string) 

Составление запроса и его выполнение 

In [ ]:
date = input('Введите последнюю дату за которую формируется отчет в формате ДД: ')
month =  input('Введите месяц за который формируется отчет в формате ММ: ')

In [ ]:
query = f'''
WITH mv_nomencl AS
 (SELECT /*+NOPARALLEL */
dmn.cd, dmn.med_nomenclature_key, dmn.nm
FROM dwh.dim_med_nomenclature dmn
WHERE dmn.cd NOT IN
         (45243, 170487, 170488, 176126, 170412, 170411, 170414, 159092,
          158854, 159093, 158853, 35160, 35159, 186229, 170486, 169758, 170485,
          166442, 166444, 178015, 166446, 77479, 50464, 19286, 74912, 21185,
          21184, 20053, 20052, 20074, 20073, 20060, 50463, 34014, 79413, 50462,
          50461, 79412, 87599, 168365, 158946, 77480, 23077, 19284, 19283,
          19285))
          
SELECT t.cd "Код аптеки",
       t.nm "Аптека",
       t.cashier "Кассир",              
       sum(ct) "Кол во позиций всего",
       count(DISTINCT(t.cheque_key)) "Кол во чеков",
       count(DISTINCT(decode(t.ct, 1, t.cheque_key, NULL))) "Кол во чеков =1 "
FROM (
SELECT dc.cd,
               dc.nm,
--ddt.department_type_nm,
f.dt_operation,
               f.cheque_key,
               dq.cashier,
               count(1) ct
FROM dwh.fact_move_goods f
JOIN dwh.dim_contragent dc ON
f.owner_key = dc.contragent_key
JOIN dwh.dim_cheque dq ON
f.cheque_key = dq.cheque_key
JOIN mv_nomencl mn ON
f.med_nomenclature_key = mn.med_nomenclature_key
JOIN dwh.dim_department dd ON
f.department_key = dd.department_key
JOIN dwh.dim_department_type ddt ON
dd.department_type_key = ddt.department_type_key
JOIN dwh.dim_med_kind dmk ON
f.med_kind_key = dmk.med_kind_key
-- where dc.cd in (52244,52279,52324,52325,52400)
AND f.operation_kind_key = 2
--операция продажи 
AND f.account_type_key = 2
-- розничная операция 
AND f.data_type = 12
-- ефарма
AND f.cheque_key IS NOT NULL
AND (f.dt_operation BETWEEN '01.{month}.2022' AND '{date}.{month}.2022')
--and dmk.cd in (229,254,255,230,257)
AND ddt.cd IN (10, 15, 16, 19, 20, 23)
-- 10 = розница, 20 = ветеринария, 16 = ортопедия, 15 = оптика материалы, 19 = оптика производство, 23 = оптика.
GROUP BY dc.cd,
                  dc.nm,
--ddt.department_type_nm,
				  f.cheque_key,
                  dq.cashier,
                  dq.cashbox_nm,
                  f.dt_operation
) t
GROUP BY t.cd,
          t.nm,
          to_char(t.dt_operation, 'yyyy_mm'),
          t.cashier
ORDER BY 1, 2, 3
    
'''

df = pd.read_sql_query(query, con=engine)

Замена пустых полей в столце "Кассир" и замена на "Администратор" через fillna

In [ ]:
df['Кассир'] = df['Кассир'].fillna('Администратор')

In [ ]:
cra = pd.read_excel(r'C:\Users\nchumurov\Desktop\ipynb\Еженедельный отчет - Количество позиций в чеке\Otchet.xlsx', usecols="A,C,Y", skiprows = 1) \
    .rename({'Unnamed: 0':'Код аптеки', 'Unnamed: 2':'ЦРА', 'Маркетинговая категория':'Категория аптеки'},axis = 1)
cra['Код аптеки'] = pd.to_numeric(cra['Код аптеки'], errors='coerce')
cra['Категория аптеки'] = cra['Категория аптеки'].fillna('Без категории')
cra = cra.dropna()
cra['Код аптеки'] = cra['Код аптеки'].astype('int64')
cra['ЦРА'] = cra['ЦРА'].apply(lambda x: x.replace('Филиал', 'ЦРА'))
cra['ЦРА'].loc[(cra['ЦРА'] == 'ЦРА 3') | (cra['ЦРА'] == 'ЦРА 300')] = 'ЦРА 3, 300'

In [ ]:
df = df.merge(cra, on='Код аптеки', how='left')


In [ ]:
df['Кассир'] = df['Кассир'].str.extract(r'([А-Я].*$)')   

In [ ]:
del_kassiri = ['123 А.В.', '123 О.Е.', '123 Л.Н.', '123 А.А.', '123 В.С.']


In [ ]:
df = df.query('Кассир not in @del_kassiri')

Чтение аналогичного отчета за прошлую неделю

In [ ]:
pharm_last_week = pd.read_csv('Итоги отчета за прошлую неделю по Фармацевтам.csv', encoding='windows-1251')
pharm_last_week

In [ ]:
category_pivot = df.pivot_table(index=['Категория аптеки'], values=['Кол во позиций всего', 'Кол во чеков', 'Кол во чеков =1 '], aggfunc='sum', margins = True, margins_name = 'Итого').reset_index()
category_pivot.columns = ['Категория аптеки','Общее кол-во позиций в чеке',  'Общее кол-во чеков', 'Кол во чеков с 1 позицией']
category_pivot['Ср. кол-во поз. в чеке ' + '1-' + date] = (category_pivot['Общее кол-во позиций в чеке'] / category_pivot['Общее кол-во чеков']).round(2)
category_pivot['Доля чеков с 1-ой поз. ' + '1-' + date] = category_pivot['Кол во чеков с 1 позицией'] / category_pivot['Общее кол-во чеков']
category_pivot

In [ ]:
category_last_week = pd.read_csv('Итоги отчета за прошлую неделю по Категории.csv', encoding='windows-1251')
category_last_week

In [ ]:
category_pivot[['Категория аптеки', 'Ср. кол-во поз. в чеке ' + '1-' + date,'Доля чеков с 1-ой поз. ' + '1-' + date]] \
    .to_csv('Итоги отчета за прошлую неделю по Категории.csv', index=False, encoding='windows-1251')

In [ ]:
category_pivot = category_pivot.merge(category_last_week, on = 'Категория аптеки')

In [ ]:
sp_column_names = category_pivot.columns.to_list()
sp_column_names[4], sp_column_names[6] = sp_column_names[6], sp_column_names[4]
sp_column_names[5], sp_column_names[7] = sp_column_names[7], sp_column_names[5]
category_pivot.columns = sp_column_names
category_pivot

In [ ]:
pivot_sp = df.pivot_table(index=['Код аптеки', 'Аптека', 'ЦРА', 'Категория аптеки'], values=['Кол во позиций всего', 'Кол во чеков', 'Кол во чеков =1 '], aggfunc='sum').reset_index()
pivot_sp.columns = ['Код аптеки', 'Аптека', 'ЦРА', 'Категория аптеки','Общее кол-во позиций в чеке', 
                    'Общее кол-во чеков', 'Кол во чеков с 1 позицией']
pivot_sp

Приводит массив данных в табличных вид в pivot_sp

In [ ]:
CRA_total = pivot_sp.groupby('ЦРА')[['Общее кол-во позиций в чеке', 'Общее кол-во чеков', 'Кол во чеков с 1 позицией']].agg('sum').reset_index()
CRA_total['ЦРА'] = CRA_total['ЦРА'] + ' итого'
CRA_total

Промежуточные итоги для ЦРА

In [ ]:
pivot_sp = pivot_sp.append(CRA_total).sort_values(['ЦРА', 'Код аптеки', 'Аптека'])
pivot_sp = pivot_sp.reset_index(drop=True)
pivot_sp

Добавление промитоги по ЦРА в pivot_sp, сортировка отчета по столбцам

In [ ]:
pivot_sp['Ср. кол-во поз. в чеке ' + '1-' + date] = (pivot_sp['Общее кол-во позиций в чеке'] / pivot_sp['Общее кол-во чеков']).round(2)
pivot_sp['Доля чеков с 1-ой поз. ' + '1-' + date] = pivot_sp['Кол во чеков с 1 позицией'] / pivot_sp['Общее кол-во чеков']
pivot_sp

In [ ]:
sp_last_week = pd.read_csv('Итоги отчета за прошлую неделю по СП.csv', encoding='windows-1251')
sp_last_week

Ввод показателей ср. кол-во позиций и доля с помощью формул

In [ ]:
pivot_sp[['Код аптеки', 'ЦРА',
          'Ср. кол-во поз. в чеке ' + '1-' + date, 
          'Доля чеков с 1-ой поз. ' + '1-' + date]].\
        to_csv('Итоги отчета за прошлую неделю по СП.csv', index=False, encoding='windows-1251')

Сохранение полученной таблицы по СП

In [ ]:
pivot_pharm = df.pivot_table(index=['Код аптеки', 'Аптека', 'Кассир'], values=['Кол во позиций всего', 'Кол во чеков', 'Кол во чеков =1 '], aggfunc='sum').reset_index()
pivot_pharm.columns = ['Код аптеки', 'Аптека', 'Кассир', 'Общее кол-во позиций в чеке', 'Общее кол-во чеков', 
                       'Кол во чеков с 1 позицией']
pivot_pharm

In [ ]:
pivot_pharm = pivot_pharm.merge(cra, on='Код аптеки', how='left')
pivot_pharm = pivot_pharm[['Код аптеки', 'Аптека', 'ЦРА', 'Кассир', 'Общее кол-во позиций в чеке', 
                           'Общее кол-во чеков', 'Кол во чеков с 1 позицией']]
pivot_pharm = pivot_pharm.sort_values(['ЦРА', 'Код аптеки', 'Аптека']).reset_index(drop=True)
pivot_pharm

In [ ]:
pivot_pharm['Ср. кол-во поз. в чеке ' + '1-' + date] = (pivot_pharm['Общее кол-во позиций в чеке'] / pivot_pharm['Общее кол-во чеков']).round(2)
pivot_pharm['Доля чеков с 1-ой поз. ' + '1-' + date] = pivot_pharm['Кол во чеков с 1 позицией'] / pivot_pharm['Общее кол-во чеков']
pivot_pharm

In [ ]:
pivot_pharm = pivot_pharm.merge(pharm_last_week, on=['Код аптеки','Кассир'], how='left')
pivot_pharm

In [ ]:
pivot_pharm[['Код аптеки', 'Кассир', 
             'Ср. кол-во поз. в чеке ' + '1-' + date, 
             'Доля чеков с 1-ой поз. ' + '1-' + date]].\
           to_csv('Итоги отчета за прошлую неделю по Фармацевтам.csv', index=False, encoding='windows-1251')

In [ ]:
pivot_sp = pivot_sp.merge(sp_last_week, on=['Код аптеки','ЦРА'], how='left')
pivot_sp

In [ ]:
sp_column_names = pivot_sp.columns.to_list()
sp_column_names[7], sp_column_names[9] = sp_column_names[9], sp_column_names[7]
sp_column_names[8], sp_column_names[10] = sp_column_names[10], sp_column_names[8]

In [ ]:
pivot_sp = pivot_sp[sp_column_names]
pivot_sp

In [ ]:
pharm_column_names = pivot_pharm.columns.to_list()
pharm_column_names[7], pharm_column_names[9] = pharm_column_names[9], pharm_column_names[7]
pharm_column_names[8], pharm_column_names[10] = pharm_column_names[10], pharm_column_names[8]

In [ ]:
pivot_pharm = pivot_pharm[pharm_column_names]
pivot_pharm

In [ ]:
file_name =f'Кол-во поз в чеке 1-{date}.{month}.2022.xlsx'
file_name

In [ ]:
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
workbook = writer.book

header_format = workbook.add_format({'bold':True,
                                     'text_wrap':True,
                                     'valign':'top',
                                     'align':'center',
                                     'fg_color':'#D7E4BC',
                                     'border':1,
                                     'font_size':9})
header_format_curr_week = workbook.add_format({'bold':True,
                                               'text_wrap':True,
                                               'valign':'top',
                                               'align':'center',
                                               'fg_color':'#7CC8D4',
                                               'border':1,
                                               'font_size':9})
table_format = workbook.add_format({'border': 1,
                                    'font_size':9})
percent_format = workbook.add_format({'border': 1,
                                      'font_size':9,
                                      'num_format':'0.0%'})
subtotal_format = workbook.add_format({'bold': True,
                                       'border':1,
                                       'font_size':9,
                                       'fg_color':'#DDB671'})
subtotal_percent_format = workbook.add_format({'bold': True,
                                               'border':1,
                                               'font_size':9,
                                               'fg_color':'#DDB671',
                                               'num_format':'0.0%'})

pivot_sp.to_excel(writer, sheet_name='СП', index=False, startrow=1, header=False)

worksheet_sp = writer.sheets['СП']

worksheet_sp.autofilter(0, 0, pivot_sp.shape[0], pivot_sp.shape[1]-1)
worksheet_sp.set_column('A:A', None, table_format)
worksheet_sp.set_column('B:B', 75, table_format)
worksheet_sp.set_column('C:C', 11, table_format)
worksheet_sp.set_column('D:D', 25, table_format)

worksheet_sp.set_column('E:H', 15, table_format)
worksheet_sp.set_column('I:I', 15, percent_format)
worksheet_sp.set_column('J:J', 15, table_format)
worksheet_sp.set_column('K:K', 15, percent_format)

for row_for_height in range(1, pivot_sp.shape[0]+1):
    worksheet_sp.set_row(row_for_height, 12)

for col_num, value in enumerate(pivot_sp.columns.values):
    if col_num in (9, 10):
        worksheet_sp.write(0, col_num, value, header_format_curr_week)
    else:
        worksheet_sp.write(0, col_num, value, header_format)
    
for row_num in pivot_sp[pivot_sp['ЦРА'].str.find('итог') > 0].index.values:
    row = pivot_sp.loc[row_num,]
    for col_num, value in enumerate(row.values):
        try:
            if col_num in (8, 10):
                worksheet_sp.write(row_num+1, col_num, value, subtotal_percent_format)
            else:
                worksheet_sp.write(row_num+1, col_num, value, subtotal_format)
        except:
            worksheet_sp.write(row_num+1, col_num, '', subtotal_format)



category_pivot.to_excel(writer, sheet_name='СП', index=False, startrow=pivot_sp.shape[0]+4, startcol=3, header=False)            


for col_num, value in enumerate(category_pivot.columns):
    col_num = col_num + 3
    worksheet_sp.write(pivot_sp.shape[0] + 3, col_num, value, header_format)


for col_num, value in enumerate(category_pivot.loc[category_pivot[category_pivot['Категория аптеки'].str.find('Итого') >= 0].index[0], ]):
        col_num = col_num + 3
        try:
            if col_num in (8, 10):
                worksheet_sp.write(pivot_sp.shape[0] + 4 + category_pivot[category_pivot['Категория аптеки'].str.find('Итого') >= 0].index[0], 
                                   col_num, value, subtotal_percent_format)
            else:
                worksheet_sp.write(pivot_sp.shape[0] + 4 + category_pivot[category_pivot['Категория аптеки'].str.find('Итого') >= 0].index[0],
                                   col_num, value, subtotal_format)
        except:
            worksheet_sp.write(pivot_sp.shape[0] + 4 + category_pivot[category_pivot['Категория аптеки'].str.find('Итого') >= 0].index[0],
                               col_num, '', subtotal_format)



for row_for_height in range(1, pivot_sp.shape[0] + 4 + category_pivot.shape[0] + 1):
    if row_for_height == 1 or row_for_height == pivot_sp.shape[0] + 3:
        pass
    else:
        worksheet_sp.set_row(row_for_height, 12)         

pivot_pharm.to_excel(writer, sheet_name='Фармацевты', index=False, startrow=1, header=False)


worksheet_pharm = writer.sheets['Фармацевты']

for row_for_height in range(1, pivot_pharm.shape[0]+1):
    worksheet_pharm.set_row(row_for_height, 12)

worksheet_pharm.autofilter(0, 0, pivot_pharm.shape[0], pivot_pharm.shape[1]-1)
worksheet_pharm.set_column('A:A', None, table_format)
worksheet_pharm.set_column('B:B', 75, table_format)
worksheet_pharm.set_column('C:C', None, table_format)
worksheet_pharm.set_column('D:D', 20, table_format)
worksheet_pharm.set_column('E:H', 15, table_format)
worksheet_pharm.set_column('I:I', 15, percent_format)
worksheet_pharm.set_column('J:J', 15, table_format)
worksheet_pharm.set_column('K:K', 15, percent_format)

for col_num, value in enumerate(pivot_pharm.columns.values):
    if col_num in (9, 10):
        worksheet_pharm.write(0, col_num, value, header_format_curr_week)
    else:
        worksheet_pharm.write(0, col_num, value, header_format)
    
writer.save()